In [12]:
import torch
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

import os
import numpy as np
from typing import List, Tuple

In [39]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 64

INPUT_SIZE = 3
HIDDEN_SIZE = 128
NUM_LAYERS = 2

LEARNING_RATE = 0.003
EPOCHS = 10

In [29]:
# Torch dataloader
class MovementsDataset(Dataset):
    
    NUMBER_OF_JOINTS = 25
    NUMBER_OF_AXES = 3
    
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        
        self.data_files = list(sorted(os.listdir(self.root)))
        self.file_frames: List[int] = []

        self.classes = dict()
        
        # Load number of frames for every file
        for fn in self.data_files:
            with open(os.path.join(self.root, fn)) as f:
                header = f.readline().split()[-1].split("_")
                
                self.file_frames.append(int(header[-1]))  # last element - number_of_frames
                
                label = int(header[1])  # second element - label
                if label not in self.classes:
                    self.classes[label] = len(self.classes) + 1
        
    def _get_file_index(self, frame_indx) -> Tuple[int, int]:
        start_indx = frame_indx
        for i, nof in enumerate(self.file_frames):
            if start_indx < nof:
                # print(f"{start_indx} - {i}")
                return i, start_indx
            else:
                start_indx -= nof
        
    def __getitem__(self, indx):
        file_indx, line_indx = self._get_file_index(indx)
        action_file = os.path.join(self.root, self.data_files[file_indx])
        
        with open(action_file, "r") as f:
            data_str = f.read().rstrip('\n').split('\n')
        
        line_indx += 2  # first two header lines in the file   
        frame = np.array([triple.split(", ") for triple in data_str[line_indx].split(";")], dtype=np.float32)
        assert frame.shape == (self.NUMBER_OF_JOINTS, self.NUMBER_OF_AXES)
        
        target = self.classes[int(data_str[0].split()[-1].split("_")[1])]
        
        if self.transforms:
            frame = self.transforms(frame)
        
        return frame, target
    
    def __len__(self) -> int:
        return len(self.data_files)

In [36]:
class BiRNN(nn.Module):

    def __init__(self, input_size: int, hidden_size: int, num_layers: int, num_classes: int):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size).to(device)

        # Forward to LSTM
        out, _ = self.lstm(x, (h0, c0))  # output format: (batch_size, seq_length, hidden_size * 2)

        out = self.fc(out[:, -1, :])
        return out

In [30]:
md = MovementsDataset(
    "../data/exports/actions-single-subject-all-POS_2020-04-29T15:15:27.384264",
    transforms=transforms.ToTensor()
)

In [31]:
train_loader = DataLoader(md, batch_size=BATCH_SIZE, shuffle=True)

In [32]:
md.classes

{10: 1,
 11: 2,
 15: 3,
 19: 4,
 20: 5,
 22: 6,
 23: 7,
 29: 8,
 31: 9,
 32: 10,
 38: 11,
 39: 12,
 41: 13,
 42: 14,
 43: 15,
 44: 16,
 45: 17,
 4: 18,
 8: 19,
 9: 20,
 13: 21,
 17: 22,
 1: 23,
 25: 24,
 2: 25,
 30: 26,
 33: 27,
 34: 28,
 36: 29,
 37: 30,
 3: 31,
 40: 32,
 46: 33,
 48: 34,
 49: 35,
 50: 36,
 51: 37,
 5: 38,
 6: 39,
 7: 40,
 47: 41,
 35: 42,
 28: 43}

In [40]:
# Training the network
model = BiRNN(INPUT_SIZE, HIDDEN_SIZE, NUM_LAYERS, len(md.classes))

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Train the model
for epoch in range(EPOCHS):
    for i, (sequnces, labels) in enumerate(train_loader):
        sequnces = sequnces.reshape(-1, 25, 3).to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(sequnces)

        loss = criterion(outputs, labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if i % 49 == 0:
            print(f"Epoch [{epoch + 1}/{EPOCHS}], Loss: {round(loss.item(), 4)}")

print("DONE")

Epoch [1/10], Loss: 3.762
Epoch [1/10], Loss: 3.3008
Epoch [1/10], Loss: 2.7687
Epoch [1/10], Loss: 2.1449
Epoch [2/10], Loss: 2.1568
Epoch [2/10], Loss: 1.7988
Epoch [2/10], Loss: 1.8185
Epoch [2/10], Loss: 1.4929
Epoch [3/10], Loss: 1.5943
Epoch [3/10], Loss: 1.4505
Epoch [3/10], Loss: 1.4163
Epoch [3/10], Loss: 1.1005
Epoch [4/10], Loss: 1.2904
Epoch [4/10], Loss: 0.8053
Epoch [4/10], Loss: 0.8002
Epoch [4/10], Loss: 0.9224
Epoch [5/10], Loss: 0.8556
Epoch [5/10], Loss: 0.9158
Epoch [5/10], Loss: 0.8475
Epoch [5/10], Loss: 0.7226
Epoch [6/10], Loss: 0.5301
Epoch [6/10], Loss: 0.5021
Epoch [6/10], Loss: 0.5364
Epoch [6/10], Loss: 0.6056
Epoch [7/10], Loss: 0.4898
Epoch [7/10], Loss: 0.5369
Epoch [7/10], Loss: 0.4403
Epoch [7/10], Loss: 0.3607
Epoch [8/10], Loss: 0.2511
Epoch [8/10], Loss: 0.2902
Epoch [8/10], Loss: 0.3301
Epoch [8/10], Loss: 0.2539
Epoch [9/10], Loss: 0.2966
Epoch [9/10], Loss: 0.245
Epoch [9/10], Loss: 0.226
Epoch [9/10], Loss: 0.1766
Epoch [10/10], Loss: 0.1722
Epo

In [41]:
# Test model
with torch.no_grad():
    correct = 0
    total = 0
    for sequences, labels in train_loader:
        sequences = sequences.reshape(-1, 25, 3).to(device)
        labels = labels.to(device)
        
        outputs = model(sequences)
        _, predicted = torch.max(outputs.data, dim=1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total}")

# Save model
torch.save(model.state_dict(), 'model.pth')

Test Accuracy: 95.48435923309788


In [ ]:
# TODO
#  - rozdelit na trenovacie/testovanie 80/20
#  - pocet epoch nastavit na 20